In [29]:
import pandas as pd
import numpy as np
import os
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

from lib.io import load_pickle_data
from lib.noglobal import noglobal
from lib.kalman_filter import generate_kalmanfilter,apply_kalmanfilter

from external_lib.gnss_transformer import get_gnss_data_path_from_phone,load_gnss_data_from_file
from external_lib.visualize import visualize_trafic,get_googlemap_url
from external_lib.gnss_manager import calc_acce_direction
import external_lib.constant as constants
from external_lib.evaluation_function import calc_haversine


In [2]:
train_df_path =  "/work/data/input/selfmade_dataset/baseline_with_derived_data_v4/train.pkl"
train_df = load_pickle_data(train_df_path)
train_df = train_df.rename(columns={"millisSinceGpsEpoch":"MillisSinceGpsEpoch"})

test_df_path =  "/work/data/input/selfmade_dataset/baseline_with_derived_data_v4/test.pkl"
test_df = load_pickle_data(test_df_path)
test_df = test_df.rename(columns={"millisSinceGpsEpoch":"MillisSinceGpsEpoch"})

## train data

### merge points実行

In [15]:
list_ = ["2021-01-05-US-SVL-1_Mi8","2021-04-26-US-SVL-1_Mi8"]
s = [];

train_result = merge_start_end_points(train_df,merge_flag=True,kalman_flag=False);

for key,each_df in tqdm(train_df.groupby("phone")):    
    
    s.append(calc_haversine(each_df["latDeg"],each_df["lngDeg"],each_df["latDeg_gt"],each_df["lngDeg_gt"]).mean())
        
                
print("実施前",calc_haversine(train_df["latDeg"],train_df["lngDeg"],train_df["latDeg_gt"],train_df["lngDeg_gt"]).mean())         
print("実施後",calc_haversine(train_result["latDeg"],train_result["lngDeg"],train_df["latDeg_gt"],train_df["lngDeg_gt"]).mean())         

100%|██████████| 73/73 [04:57<00:00,  4.08s/it]


  0%|          | 0/73 [00:00<?, ?it/s]

実施前 3.8468553394627825
実施後 3.798292363187363


### sample visualize

#### 実行前

In [19]:
sample_path = "2020-06-04-US-MTV-1_Pixel4";

tmp_df = train_df[train_df["phone"] == sample_path];
visualize_trafic(tmp_df)

#### 実行後

In [30]:
sample_path = "2020-06-04-US-MTV-1_Pixel4";

tmp_df = train_result[train_result["phone"] == sample_path];
visualize_trafic(tmp_df)


get_googlemap_url(tmp_df[["latDeg","lngDeg"]].iloc[:1])

https://www.google.co.jp/maps/@37.41633007021277,-122.08047291063833,14z?hl=ja


'https://www.google.co.jp/maps/@37.41633007021277,-122.08047291063833,14z?hl=ja'

#### 正解データ

In [25]:
sample_path = "2020-06-04-US-MTV-1_Pixel4";

tmp_df = train_result[train_result["phone"] == sample_path];
tmp_df[["latDeg","lngDeg"]]=tmp_df[["latDeg_gt","lngDeg_gt"]]
visualize_trafic(tmp_df)

## test

### submission.csv用データ作成

In [7]:
from external_lib.merge_start_end_points import merge_start_end_points
submission_flag = False

if (submission_flag):
    test_df_path =  "/work/data/input/selfmade_dataset/baseline_with_derived_data_v4/test.pkl"
    test_df = load_pickle_data(test_df_path)


    test_result = merge_start_end_points(test_df,merge_flag=True,kalman_flag=False);
    
    print("変化量",calc_haversine(test_result["latDeg"],test_result["lngDeg"],test_df["latDeg"],test_df["lngDeg"]).mean())


    sub_path = "/work/data/input/google-smartphone-decimeter-challenge/sample_submission.csv"
    sub = pd.read_csv(sub_path);

    sub["latDeg"] = test_result["latDeg"]
    sub["lngDeg"] = test_result["lngDeg"]
    display(sub.loc[25:35])

    if (not os.path.exits("/work/data/to_csv.csv")):
        sub.to_csv("/work/data/to_csv.csv",index=False);

### グラフ表示

#### merge前

In [10]:
if (submission_flag):
    path = "2020-05-15-US-MTV-1_Pixel4"
    each_df = sub[sub["phone"] == path]
    display(visualize_trafic(each_df))

In [12]:
if (submission_flag):
    path = "2020-05-15-US-MTV-1_Pixel4"
    each_df = test_df[test_df["phone"] == path]
    display(visualize_trafic(each_df))